## Feature/X-Label Building by extracting words from Text File 

In [3]:
# importing stuff
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
plt.rcParams['figure.figsize']=(10,6)
import glob
from string import punctuation
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
import operator
from scipy.sparse import csr_matrix

In [4]:
#Function to Check is it number
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [549]:
# function to read the file and return a list of words
def parse_text_tolist(path):
    try:
        #read file
        soup=open(path,'r')
        #strip punctuations
        raw_text=strip_pnctn(soup)
        #Split words
        word_list = raw_text.split()
        words_list=[]
        for i in range(len(word_list)):
            chars = ''.join(s for s in word_list[i] if s.isalnum()) #splitting into chars for aplhanumeric
            if is_number(chars)==False: #removing numbers
                words_list.append(chars)
    except ValueError:
        print("value Error")
    #Convert text to lower case inorder to avoid reading same word again and again 
    words_list= [x.lower() for x in words_list]
    return words_list

In [778]:
# function to get sorted freq from each class
def getSortedFreq(classPath):
    #Class Path is the folder in OS where all the files to be read is located
    path_list=glob.glob(classPath)    
    path_list_100= path_list[600:700] #take the 100 file names in list to read the contents iteratively 
    words_lists=[]
    freqs_lists=[]
    for i in range(len(path_list_100)):
        #freq_list=[]
        #Call the function to convert text file to word frequency list
        word_list= parse_text_tolist(path_list_100[i])
        words_lists.append(word_list)
        for j in range(len(words_lists)):
            freq = freqs(words_lists[j])
            freqs_lists.append(freq)

    # merging all values for a key and forming a new dict of key value pairs
    #from collections import defaultdict
    freqs_final = defaultdict(list)
    for i in range(len(freqs_lists)):
        for k,v in freqs_lists[i].items():
            freqs_final[k].append(v)


    # In[ ]:

    #summing all values per key in the freqs_final
    freqs_final = {k: sum_list(v) for k, v in freqs_final.items()}
    remove_keys(common_keys,freqs_final)
    sorted_freqs= sorted(freqs_final.items(),key=operator.itemgetter(1),reverse= True)
    return sorted_freqs, freqs_final

In [779]:
# function to parse the text ignoring the chars
def strip_pnctn(txt):
    return ','.join(c for c in txt if c not in ignoreList)

In [780]:
def freqs(wl):
    wordfreq = []
    wordname=[]
    #Converted list to avoid duplicates to iterate and count words
    for w in set(wl):
        wordfreq.append(wl.count(w))
        wordname.append(w)
    #word frequency
    freq=dict(zip(wordname,wordfreq))
    return freq

In [781]:
def sum_list(values):
    v= sum(values)
    return v

In [782]:
#function to remove the common keys
def remove_keys(common_keys, dicti):
    for key in common_keys:
        if key in dicti:
            del dicti[key]

In [783]:
#Function to get stop words
def getStopWords():
    with open('terrier-stop.txt', 'r') as f:
        stopwords1 = [x.strip('\n') for x in f.readlines()]

    with open('minimal-stop.txt', 'r') as f:
        stopwords2 = [x.strip('\n') for x in f.readlines()]

    with open('stoplist.txt', 'r') as f:
        stopwords3 = [x.strip('\n') for x in f.readlines()]

    stopwords4 = stopwords.words('english')

    stpwrd_full = stopwords1 +stopwords2 +stopwords3+stopwords4 

    mylist_stopwords = ["newsgroups","messageid",'gmt','path','subject','sender','lines','organization','articleid','apr','date','writes','pp','cantaloupesrvcscmuedu','xref','nntppostinghost','article','references']
#    common_keys= list(set(keys)&set(stpwrd_full)) + mylist_stopwords #mega keywords list
    common_keys= list(set(stpwrd_full)) + mylist_stopwords #mega keywords list 
    return common_keys

In [784]:
def getDictOfWords(path):
    words_list = parse_text_tolist(path)
    freq = freqs(words_list)

    # merging all values for a key and forming a new dict of key value pairs
    from collections import defaultdict
    freqs_final = defaultdict(list)
    for k,v in freq.items():
        freqs_final[k].append(v)


    freqs_final = {k: sum_list(v) for k, v in freqs_final.items()}
    return freqs_final

In [785]:
def getDatapoint(dict_of_features, dict_of_newfile):
    d_feature_list = dict_of_features
    d_inputfile = dict_of_newfile

    row_loc = []
    row_val = []
    it = 0
    for key, value in d_feature_list.items():
        if key in d_inputfile:
            row_loc.append(it)
            d_feature_list[key] = d_inputfile[key]
            row_val.append(d_feature_list[key])
        else:
            d_feature_list[key] = 0
        it = it + 1
    return row_loc, row_val

In [786]:
###global part of the code
common_keys = getStopWords()
# creating a list of chars to be ignored
num=['0','1','2','3','4','5','6','7','8','9','--']
ignoreList= punctuation + "1 2 3 4 5 6 7 8 9 0"

In [787]:
# 1.-------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (ATHEISM)
#-------------------------------------------------------------------------------------------------------------

# Lets read the file directories and store the path
path_list_atheism = "E:\\Sem 2\\20_newsgroups\\alt.atheism\\*"
sorted_freqs, freqs_final = getSortedFreq(path_list_atheism)

In [789]:
#2. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (RELIGION MISC)
#-------------------------------------------------------------------------------------------------------------
path_list_RelMisc ="E:\\Sem 2\\20_newsgroups\\talk.religion.misc\\*"
sorted_freqs_RelMisc, dummy = getSortedFreq(path_list_RelMisc)

In [790]:

#3. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (REL CHRISTIAN)
#-------------------------------------------------------------------------------------------------------------
path_list_RelChrist ="E:\\Sem 2\\20_newsgroups\\soc.religion.christian\\*"
sorted_freqs_RelChrist, dummy = getSortedFreq(path_list_RelChrist)

In [791]:
#4.-------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (POLITICS GUNS)
#-------------------------------------------------------------------------------------------------------------

path_list_Guns ="E:\\Sem 2\\20_newsgroups\\talk.politics.guns\\*"
sorted_freqs_guns, dummy = getSortedFreq(path_list_Guns)

In [792]:

#5. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (POLITICS MIDEAST)
#-------------------------------------------------------------------------------------------------------------
path_list_MidEast ="E:\\Sem 2\\20_newsgroups\\talk.politics.mideast\\*"
sorted_freqs_MidEast, dummy = getSortedFreq(path_list_MidEast)

In [793]:
#6.-------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (POLITICS MISC)
#-------------------------------------------------------------------------------------------------------------
path_list_PolMisc ="E:\\Sem 2\\20_newsgroups\\talk.politics.misc\\*"
sorted_freqs_PolMisc, dummy = getSortedFreq(path_list_PolMisc)

In [794]:

#7.-------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (SCI SPACE)
#-------------------------------------------------------------------------------------------------------------
path_list_SciSpace ="E:\\Sem 2\\20_newsgroups\\sci.space\\*"
sorted_freqs_SciSpace, dummy = getSortedFreq(path_list_SciSpace)

In [795]:


#8. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (SCI MED)
#-------------------------------------------------------------------------------------------------------------
path_list_SciMed ="E:\\Sem 2\\20_newsgroups\\sci.med\\*"
sorted_freqs_SciMed, dummy = getSortedFreq(path_list_SciMed)

In [796]:
#9. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (SCI ELECTRONICS)
#-------------------------------------------------------------------------------------------------------------
path_list_SciElec ="E:\\Sem 2\\20_newsgroups\\sci.electronics\\*"
sorted_freqs_SciElec, dummy = getSortedFreq(path_list_SciElec)

In [797]:
#10. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (SCI CRYPT)
#-------------------------------------------------------------------------------------------------------------
path_list_SciCrypt ="E:\\Sem 2\\20_newsgroups\\sci.crypt\\*"
sorted_freqs_SciCrypt, dummy = getSortedFreq(path_list_SciCrypt)

In [798]:
#11. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (BASEBALL)
#-------------------------------------------------------------------------------------------------------------
path_list_Baseball ="E:\\Sem 2\\20_newsgroups\\rec.sport.baseball\\*"
sorted_freqs_Baseball, dummy = getSortedFreq(path_list_Baseball)

In [799]:
#12. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (MOTORCYCLE)
#-------------------------------------------------------------------------------------------------------------
path_list_Motor ="E:\\Sem 2\\20_newsgroups\\rec.motorcycles\\*"
sorted_freqs_Motor, dummy = getSortedFreq(path_list_Motor)

In [800]:

#13. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (AUTOS)
#-------------------------------------------------------------------------------------------------------------
path_list_Autos ="E:\\Sem 2\\20_newsgroups\\rec.autos\\*"
sorted_freqs_Autos, dummy = getSortedFreq(path_list_Autos)

In [801]:

#14. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (HOCKEY)
#-------------------------------------------------------------------------------------------------------------
path_list_Hockey="E:\\Sem 2\\20_newsgroups\\rec.sport.hockey\\*"
sorted_freqs_hockey, dummy = getSortedFreq(path_list_Hockey)

In [802]:
#15. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (FORSALE)
#-------------------------------------------------------------------------------------------------------------
path_list_ForSale ="E:\\Sem 2\\20_newsgroups\\misc.forsale\\*"
sorted_freqs_ForSale, dummy = getSortedFreq(path_list_ForSale)

In [804]:

#16.-------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (WINDOWS)
#-------------------------------------------------------------------------------------------------------------
path_list_Wind ="E:\\Sem 2\\20_newsgroups\\comp.windows.x\\*"
sorted_freqs_Wind, dummy = getSortedFreq(path_list_Wind)

In [805]:
#17. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (HARDWARE)
#-------------------------------------------------------------------------------------------------------------
path_list_Hardware ="E:\\Sem 2\\20_newsgroups\\comp.sys.mac.hardware\\*"
sorted_freqs_Hardware, dummy = getSortedFreq(path_list_Hardware)

In [806]:
#18. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (PC HARDWARE)
#-------------------------------------------------------------------------------------------------------------
path_list_PCHardware ="E:\\Sem 2\\20_newsgroups\\comp.sys.ibm.pc.hardware\\*"
sorted_freqs_PCHardware, dummy = getSortedFreq(path_list_PCHardware)

In [807]:
#19. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (MISC WINDOWS)
#-------------------------------------------------------------------------------------------------------------
path_list_MiscWind ="E:\\Sem 2\\20_newsgroups\\comp.os.ms-windows.misc\\*"
sorted_freqs_MiscWind, dummy = getSortedFreq(path_list_MiscWind)

In [808]:
#20. -------------------------------------------------------------------------------------------------------------
# # Let us read our next set of files and repeat the procedure (COMP GRAPHICS)
#-------------------------------------------------------------------------------------------------------------
path_list_Graphics ="E:\\Sem 2\\20_newsgroups\\comp.graphics\\*"
sorted_freqs_Graphics, dummy = getSortedFreq(path_list_Graphics)

In [809]:
#-------------------------------------------------------------------------------------------------------------
# # MERGE and get mega features
#-------------------------------------------------------------------------------------------------------------

# Now that we have features from different classes, we will merge them by picking 30/50/100 from each
mega_feature_list = []
topn = 30
# picking the top  features
athisem_features = {k: v for k, v in sorted_freqs[:topn]} 
relmisc_features = {k: v for k, v in sorted_freqs_RelMisc[:topn]}
relChrist_features = {k: v for k, v in sorted_freqs_RelChrist[:topn]}
guns_features = {k: v for k, v in sorted_freqs_guns[:topn]} 
mideast_features=  {k: v for k, v in sorted_freqs_MidEast[:topn]}
polmisc_features = {k: v for k, v in sorted_freqs_PolMisc[:topn]}
scispace_features = {k: v for k, v in sorted_freqs_SciSpace[:topn]}
scimed_features = {k: v for k, v in sorted_freqs_SciMed[:topn]}
scielec_features = {k: v for k, v in sorted_freqs_SciElec[:topn]}
scicrypt_features = {k: v for k, v in sorted_freqs_SciCrypt[:topn]}
baseball_features = {k: v for k, v in sorted_freqs_Baseball[:topn]}
motorcycle_features = {k: v for k, v in sorted_freqs_Motor[:topn]}
autos_features = {k: v for k, v in sorted_freqs_Autos[:topn]}
hockey_features = {k:v for k,v in sorted_freqs_hockey[:topn]}
forsale_features = {k: v for k, v in sorted_freqs_Autos[:topn]}
windows_features = {k: v for k, v in sorted_freqs_Wind[:topn]}
hardware_features = {k: v for k, v in sorted_freqs_Hardware[:topn]}
pchardware_features = {k: v for k, v in sorted_freqs_PCHardware[:topn]}
miscwindows_features = {k: v for k, v in sorted_freqs_MiscWind[:topn]}
compgraphics_features = {k: v for k, v in sorted_freqs_Graphics[:topn]}


In [811]:
#merging the feature lists
mega_feature_list.extend((athisem_features,relmisc_features,relChrist_features,\
guns_features,mideast_features, polmisc_features,scispace_features, \
scimed_features, scielec_features, scicrypt_features,\
baseball_features, motorcycle_features, autos_features, hockey_features,\
forsale_features, windows_features, hardware_features,pchardware_features,\
miscwindows_features, compgraphics_features\
)) 

In [812]:
#creating the dictionary with adding up duplicates
mega_features_final= defaultdict(list) 
for i in range(len(mega_feature_list)):
    for k,v in mega_feature_list[i].items():
        mega_features_final[k].append(v)

mega_features_final = {k: sum_list(v) for k, v in mega_features_final.items()} #summing the counts 

sorted_mega_features= sorted(mega_features_final.items(),key=operator.itemgetter(1),reverse= True) #sorted

print(len(sorted_mega_features))
#Futher tuning by poping highest frequency word, which is " "
sorted_mega_features.pop(0)
n_features = len(sorted_mega_features)

d_mega_features = dict(sorted_mega_features)


361


In [814]:
#Putting into a DataFrame to export to CSV.
final=pd.DataFrame(sorted_mega_features)

,0,1
0,university,49540
1,dont,44561
2,people,44125
3,im,28629
4,file,24456
5,time,21878
6,system,18434
7,god,16425
8,government,15302
9,car,13473


In [815]:
final.to_csv("C:\\Users\\rajag\\Documents\\Python Scripts\\Data Sci Project\\t7.csv")

### Now the feature vector is build we can populate the data in Build Data set python file